In [ ]:
!pip install pandas
!pip install pyspark
!pip install pyarrow

In [ ]:
from pyspark.sql import SparkSession
 
# Building the SparkSession and name 
# it :'pandas to spark'
spark = SparkSession.builder.appName(
  "pandas to spark").getOrCreate()

spark.conf.set("spark.sql.execution.arrow.pyspark.enabled", "true")

In [5]:
import pandas as pd

data = [[1, 10, 5, 10, 10], [2, 20, 20, 20, 20], [3, 10, 30, 20, 20], [4, 10, 40, 40, 40]]
insurance = pd.DataFrame(data, columns=['pid', 'tiv_2015', 'tiv_2016', 'lat', 'lon']).astype({'pid':'Int64', 'tiv_2015':'Float64', 'tiv_2016':'Float64', 'lat':'Float64', 'lon':'Float64'})

In [6]:
insurance = spark.createDataFrame(insurance)
insurance.show()

+---+--------+--------+----+----+
|pid|tiv_2015|tiv_2016| lat| lon|
+---+--------+--------+----+----+
|  1|    10.0|     5.0|10.0|10.0|
|  2|    20.0|    20.0|20.0|20.0|
|  3|    10.0|    30.0|20.0|20.0|
|  4|    10.0|    40.0|40.0|40.0|
+---+--------+--------+----+----+



In [23]:
from pyspark.sql.functions import count, col, sum, round

investment = insurance \
            .groupby('tiv_2015') \
            .agg(count('pid').alias('count')) \
            .where('count > 1')

location = insurance \
            .groupby(['lat', 'lon']) \
            .agg(count('pid').alias('count')) \
            .where('count == 1')

report = insurance \
         .where(col('tiv_2015').isin(investment.toPandas()['tiv_2015'].to_list()) \
                & col('lat').isin(location.toPandas()['lat'].to_list()) \
                & col('lon').isin(location.toPandas()['lon'].to_list())   ) \
         .agg(round(sum('tiv_2016'), 2).alias('tiv_2016'))

report.show()

+--------+
|tiv_2016|
+--------+
|    45.0|
+--------+

